In [1]:
%pylab inline
import json
import numpy as np
import pandas as pd
import glob



Populating the interactive namespace from numpy and matplotlib


In [2]:
output_dir = "../results-anonymized/experiment/"
files = glob.glob(output_dir+'block_*.json')

blocks = []
for file in files:
    with open(file) as f:
        blocks.append(json.load(f))

len(blocks)

30

In [37]:
collector = []
drops = []
for block in blocks:
    drops += [len([i for i in block[list(block.keys())[j]]['log'] if i['name'] == 'drop']) for j in [0,1]]
    for game_id in block.keys():
        for player_id, player in block[game_id]['players'].items():
            info = dict()
            info['treatment'] = player['data.position'][0]
            info['caveman'] = "caveman" in game_id
            info['idled'] = len(player['data.log'])>0
            info['adoptions'] = len(player['data.notebooks']['promising_leads']['clueIDs'])
            info['rejections'] = len(player['data.notebooks']['dead_ends']['clueIDs'])
            
            
            info['completed'] = "data.caseMade" in player
            if "data.caseMade" in player:
                info['confidence'] = float(player['data.caseMade']['confidence'])
                info['max_assessment'] = np.max([float(v) for k, v in player['data.caseMade'].items() if k not in ["confidence", "consensus"]])
                info['min_assessment'] = np.min([float(v) for k, v in player['data.caseMade'].items() if k not in ["confidence", "consensus"]])
                info.update(player['data.caseMade'])
            else:
                info['confidence'] = 0
                info['max_assessment'] = 50
                info['min_assessment'] = 50
                
            if 'data.survey' in player:
                info.update(player['data.survey'])
            collector.append(info)
            
    
df = pd.DataFrame(collector)
#df.fillna("", inplace=True)

In [25]:
# number of classifications made, on average, per player
np.mean(drops)/2/20

28.42875

In [40]:
# number of adoptions made on average
df['adoptions'].mean()

16.15875

In [25]:
df['suspect_diff'] = df[['suspect_1', 'suspect_2', 'suspect_3']].max(axis=1).astype(float) - df[['suspect_1', 'suspect_2', 'suspect_3']].min(axis=1).astype(float)
df['appearance_diff'] = df[['appearance_1', 'appearance_2']].max(axis=1).astype(float) - df[['appearance_1', 'appearance_2']].min(axis=1).astype(float)
df['clothing_diff'] = df[['clothing_1', 'clothing_2']].max(axis=1).astype(float) - df[['clothing_1', 'clothing_2']].min(axis=1).astype(float)
df['tool_diff'] = df[['tool_1', 'tool_2']].max(axis=1).astype(float) - df[['tool_1', 'tool_2']].min(axis=1).astype(float)
df['vehicle_diff'] = df[['vehicle_1', 'vehicle_2']].max(axis=1).astype(float) - df[['vehicle_1', 'vehicle_2']].min(axis=1).astype(float)

In [50]:
df[['suspect_diff', 'appearance_diff', 'clothing_diff', 'tool_diff', 'vehicle_diff']].mean()

suspect_diff       48.186317
appearance_diff    30.101979
clothing_diff      31.604131
tool_diff          32.669105
vehicle_diff       40.978916
dtype: float64

In [38]:
df[(df['treatment'] == 't') & (df['caveman'] == False)][['suspect_diff', 'appearance_diff', 'clothing_diff', 'tool_diff', 'vehicle_diff']].mean()

suspect_diff       46.470690
appearance_diff    27.365517
clothing_diff      25.650000
tool_diff          29.048276
vehicle_diff       37.029310
dtype: float64

In [39]:
df[(df['treatment'] == 'c') & (df['caveman'] == False)][['suspect_diff', 'appearance_diff', 'clothing_diff', 'tool_diff', 'vehicle_diff']].mean()

suspect_diff       46.283993
appearance_diff    31.055077
clothing_diff      32.512909
tool_diff          29.082616
vehicle_diff       40.848537
dtype: float64

In [106]:
def group_genders(s):
    s = str(s).lower().strip().replace(" ","").replace("-","")
    if (s in ["female", "f", "woman", "ciswoman"] or
        "fem" in s):
        return "f"
    elif (s in ["male", "m", "man"]):
        return "m"
    elif ("prefernot" in s):
        return ""
    elif (s in ["nonbinary", "n"]):
        return "nb"
    else:
        return s
df['gender'] = df['gender'].apply(group_genders)
df['gender'].value_counts()

m          1190
f          1078
            106
nb            9
ftm           2
1             2
mal           2
mae           2
gender        1
make          1
malw          1
42            1
msle          1
agender       1
mali          1
21            1
2             1
Name: gender, dtype: int64

In [107]:
df['age'] = df['age'].apply(lambda s: int(s) if len(s)>0 else np.nan)
df['age'] = df['age'].apply(lambda s: s if 10<s<100 else np.nan)

In [108]:
df['age'].mean()

37.146214099216714

In [109]:
df['age'].median()

35.0

In [110]:
df['education'].value_counts()/2400

bachelor       0.487917
high-school    0.269583
master         0.161667
               0.047917
other          0.032917
Name: education, dtype: float64

In [111]:
df['completed'].mean()

0.9683333333333334

In [112]:
df['idled'].mean() 

0.27291666666666664

In [113]:
# difference in dropout percentage by condition
dropouts = [df[(df['treatment'] == 'c') & (df['caveman'])]['completed'].mean(),
            df[(df['treatment'] == 'c') & (~df['caveman'])]['completed'].mean(),
            df[(df['treatment'] == 't') & (df['caveman'])]['completed'].mean(),
            df[(df['treatment'] == 't') & (~df['caveman'])]['completed'].mean()]
(max(dropouts) - min(dropouts)) * 100

0.33333333333332993

In [114]:
len(df[df['confidence']>75])/2400

0.21291666666666667

In [115]:
len(df[df['max_assessment']>=95])/2400

0.5020833333333333

In [36]:
len(df[(df['max_assessment']>=95) & (df['treatment'] == 't') & (df['caveman'] == False)])/600

0.5033333333333333

In [37]:
len(df[(df['max_assessment']>=95) & (df['treatment'] == 'c') & (df['caveman'] == False)])/600

0.4716666666666667

In [120]:
len(df[(df['max_assessment']>=95) & (df['treatment'] == 'c') ])/1200

0.48833333333333334

In [116]:
len(df[df['min_assessment']<=5])/2400

0.2920833333333333

In [49]:
for i, s in df['strategy'].apply(lambda s: str(s).lower()).sort_values().items():
    print(s)



tried to eliminate clues that seemed irrelevant to establishing who the criminal was.
100
15
2
2
3
3
76
? i didn't have one - i couldn't figure anything out. none of the info seemed contradictory, so i couldn't rule anything out. it says above that everything was true - which is a bit overwhelming. it didn't make sense, in the context of this being a game and not real life, that everything would be true. so i only picked a few things, as a random guess and just hoped.
a random guess? it felt like i could make a plausible case for any of the suspects. there were so many variables to consider that i honestly can't remember which one(s) steered me towards mills
a strategy game where you have partial information
absolutely could not give you an intelligent accounting. i just looked at what my other collaborators deemed important and if there were multiple hits, i added it.
addign clue togetther. i play dungeons and dragons so this is just like that 
after finding some convincing lead reg

my strategy was to think of all the crime shows i've watched and determine what they would have thought as a promising lead.
my strategy was to try and eliminate clues that other clues had made irrelevant. i had a hard time with this however, because very few clues seemed irrelevant. i also tried to focus on things where maybe there was another explanation, such as the ripped jeans and the saw. if someone used a saw, they could rip their pants. but just because the person had ripped pants and used a saw did not mean they were the suspect.
my strategy was to try and find common leads across each of my fellow detectives who i had access too. that was i figured we were each constructing the same story.
my strategy was to try and find details that related directly to how the watch was stolen first, which were the watch was in a case and was opened with a masonry drill, i assumed these to be true as they are concrete evidence not witness related.  from there i tried to find who had or was s

In [48]:
df['strategy'].apply(lambda s: str(s).lower()).sort_values()

530                                                      
1540    \ntried to eliminate clues that seemed irrelev...
230                                                   100
66                                                     15
359                                                     2
                              ...                        
1300                                                  yes
710                                                   yes
275                                                   yes
1283                                                  yes
473     your game will rely on strategy, not luck in d...
Name: strategy, Length: 2400, dtype: object